## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [2]:
dir_path = "/Users/ecem/Desktop/seperate-axis"

In [3]:
x = np.load(dir_path + "/x-10sec.npy", allow_pickle= True)

In [4]:
x.shape

(2392, 2560)

In [5]:
def converter(array):
    x= []
    for i in range(array.shape[0]):
        x.append(obspy.core.trace.Trace(array[i]))
        
        x[i].stats.sampling_rate = 256
    return np.array(x)

In [6]:
x = converter(x)


## comparison of two dataset:

Now we will to try visualize spectograms. So far we have used Obspy to visualize SCG signals. However, with obspy we are not able to plot spectograms side by side. This is important for us because we want to see spectograms of x,y and z components simultaneously for both healty and diseased people. So, from now on I will refer to those plots, for 3 components of healty and diseased (in total six plot, as "PAIR".


I have mentioned that Obpsy has a problem in side by side plotting. So I changed it source code. I have added and deleted some part to operate according to my need. ???????????

Thats why from now on we are going to use **output_spectogram** function to get values needed to plot spectogram.

These are the imports for output_spectogram function to work.

In [7]:
from obspy.imaging.cm import obspy_sequential
import math
from matplotlib import mlab
from matplotlib.colors import Normalize

In [8]:
def _nearest_pow_2(x):
    a = math.pow(2, math.ceil(np.log2(x)))
    b = math.pow(2, math.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [9]:
def output_spectrogram(data, samp_rate, per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=obspy_sequential, zorder=None, title=None,
                show=True, clip=[0.0, 1.0]):
 
    import matplotlib.pyplot as plt
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - np.array(data).mean()
    end = npts / samp_rate

    # Here we call not plt.specgram as this already produces a plot
    # matplotlib.mlab.specgram should be faster as it computes only the
    # arrays
    # XXX mlab.specgram uses fft, would be better and faster use rfft
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap, mode = 'psd')

    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]
    
    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)

    # calculate half bin width
    halfbin_time = (time[1] - time[0]) / 2.0
    halfbin_freq = (freq[1] - freq[0]) / 2.0

    # argument None is not allowed for kwargs on matplotlib python 3.3
    kwargs = {k: v for k, v in (('cmap', cmap), ('zorder', zorder))
              if v is not None}

    if log:
        # pcolor expects one bin more at the right end
        freq = np.concatenate((freq, [freq[-1] + 2 * halfbin_freq]))
        time = np.concatenate((time, [time[-1] + 2 * halfbin_time]))
        # center bin
        time -= halfbin_time
        freq -= halfbin_freq
        # Log scaling for frequency values (y-axis)
        ax.set_yscale('log')
        # Plot times
        ax.pcolormesh(time, freq, specgram, norm=norm, **kwargs)
    else:
        # this method is much much faster!
        specgram = np.flipud(specgram)
        # center bin
        extent = (time[0] - halfbin_time, time[-1] + halfbin_time,
                  freq[0] - halfbin_freq, freq[-1] + halfbin_freq)
    return [specgram, extent, end]

    



## Plotting spectogram:

f is the frequency array, containing the frequencies of every band of the fft. Which can be used as the labels for a graph

t is the time array, containing the time at which this FFT was made relative to the source signal. Again can be used for labels.

The Sxx array contains the amplitudes and is a 2d array whose shape is the length of f by the length of t.


 The purpose of a spectogram is to take the FFT of small, equal-sized time chunks. This produces a 2D fourier transform where the X axis is the start time of the time chunk and the Y axis is the energy (or power, etc.) in each frequency in that time chunk. This allows you to see how the frequency components change over time.
 
  A spectrogram is a representation of frequency over time with the addition of amplitude as a third dimension, denoting the intensity or volume of the signal at a frequency and a time.

In [10]:
def freq_average(array):
    means= []
    for i in range(array.shape[0]):
        means.append(array[i][0:40].mean())
    return np.array(means)
        

In [11]:
output_spectrogram(x[0].data, 256)[0].shape

(2048, 40)

In [12]:
freq_average(array = output_spectrogram(x[0].data, 256)[0])

array([0.00046219, 0.00065803, 0.00066728, ..., 0.01861373, 0.01935248,
       0.01980302])

In [13]:
freq_average(array = output_spectrogram(x[0].data, 256)[0]).shape

(2048,)

In [14]:
def spec_values(array):
    x_ = []
    for i in range(array.shape[0]):
        _x = output_spectrogram(array[i].data, 256)[0]

        x_.append(freq_average(array = _x))
        
        
    return np.array(x_)

In [15]:
type(x)

numpy.ndarray

In [16]:
x_ = spec_values(x)

In [17]:
x_[0]

array([0.00046219, 0.00065803, 0.00066728, ..., 0.01861373, 0.01935248,
       0.01980302])

In [18]:
print(x_.shape)


(2392, 2048)


In [19]:
columns = []
for i in range(x_.shape[1]):
    columns.append("freq_x{}".format(i))
print(columns)

['freq_x0', 'freq_x1', 'freq_x2', 'freq_x3', 'freq_x4', 'freq_x5', 'freq_x6', 'freq_x7', 'freq_x8', 'freq_x9', 'freq_x10', 'freq_x11', 'freq_x12', 'freq_x13', 'freq_x14', 'freq_x15', 'freq_x16', 'freq_x17', 'freq_x18', 'freq_x19', 'freq_x20', 'freq_x21', 'freq_x22', 'freq_x23', 'freq_x24', 'freq_x25', 'freq_x26', 'freq_x27', 'freq_x28', 'freq_x29', 'freq_x30', 'freq_x31', 'freq_x32', 'freq_x33', 'freq_x34', 'freq_x35', 'freq_x36', 'freq_x37', 'freq_x38', 'freq_x39', 'freq_x40', 'freq_x41', 'freq_x42', 'freq_x43', 'freq_x44', 'freq_x45', 'freq_x46', 'freq_x47', 'freq_x48', 'freq_x49', 'freq_x50', 'freq_x51', 'freq_x52', 'freq_x53', 'freq_x54', 'freq_x55', 'freq_x56', 'freq_x57', 'freq_x58', 'freq_x59', 'freq_x60', 'freq_x61', 'freq_x62', 'freq_x63', 'freq_x64', 'freq_x65', 'freq_x66', 'freq_x67', 'freq_x68', 'freq_x69', 'freq_x70', 'freq_x71', 'freq_x72', 'freq_x73', 'freq_x74', 'freq_x75', 'freq_x76', 'freq_x77', 'freq_x78', 'freq_x79', 'freq_x80', 'freq_x81', 'freq_x82', 'freq_x83', '

In [20]:
print(len(columns))

2048


In [21]:
def dataframe_creator(array, df = pd.DataFrame()):
    values = []
    for i in range(array.shape[0]):
        values.append(array[i][0:2048].tolist())

        
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [22]:
df = dataframe_creator(x_, 
                         df =pd.DataFrame(columns = columns))

In [23]:
#df = df.reset_index()

In [24]:
df

,freq_x0,freq_x1,freq_x2,freq_x3,freq_x4,freq_x5,freq_x6,freq_x7,freq_x8,freq_x9,...,freq_x2038,freq_x2039,freq_x2040,freq_x2041,freq_x2042,freq_x2043,freq_x2044,freq_x2045,freq_x2046,freq_x2047
0,0.000462,0.000658,0.000667,0.000676,0.000683,0.000685,0.000687,0.000686,0.000687,0.000691,...,0.009566,0.010833,0.012223,0.013661,0.015083,0.016425,0.017622,0.018614,0.019352,0.019803
0,0.000359,0.000513,0.000522,0.000531,0.000538,0.000542,0.000546,0.000552,0.000559,0.000569,...,0.005794,0.005593,0.005357,0.005115,0.004873,0.004623,0.004383,0.00415,0.003926,0.003737
0,0.000467,0.000674,0.000698,0.000725,0.000752,0.000779,0.000804,0.000824,0.000837,0.000841,...,0.006523,0.006523,0.006484,0.006415,0.006306,0.006176,0.006001,0.005789,0.005561,0.00535
0,0.000422,0.000606,0.000621,0.000636,0.000647,0.000654,0.000656,0.000653,0.000646,0.000641,...,0.008027,0.0085,0.008899,0.009197,0.009377,0.009434,0.009375,0.009219,0.008999,0.008765
0,0.000379,0.00054,0.000547,0.000553,0.000556,0.000557,0.000559,0.000569,0.000589,0.000613,...,0.007626,0.008042,0.008359,0.008565,0.008656,0.008636,0.008516,0.008321,0.008084,0.007861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.000161,0.00023,0.000234,0.000237,0.00024,0.000243,0.000246,0.000249,0.00025,0.00025,...,0.020132,0.02292,0.02553,0.027868,0.029858,0.031441,0.032587,0.033298,0.033609,0.033594
0,0.00014,0.000202,0.000208,0.000214,0.000221,0.000227,0.000233,0.000237,0.000242,0.000248,...,0.029185,0.032617,0.035737,0.038437,0.040636,0.042283,0.043365,0.043915,0.044017,0.043826
0,0.000153,0.000218,0.000222,0.000226,0.00023,0.000234,0.000239,0.000244,0.000248,0.000251,...,0.037794,0.042437,0.046711,0.050472,0.053606,0.056039,0.057742,0.058741,0.059118,0.05903
0,0.000153,0.000221,0.00023,0.000242,0.000255,0.000269,0.000281,0.000291,0.000297,0.0003,...,0.031796,0.036175,0.04034,0.044157,0.047506,0.050294,0.052461,0.053986,0.054892,0.055269


In [25]:
df.to_csv(dir_path + '/x-axis-spectogram_coef.csv')